In [1]:
import pandas as pd
import gzip
from collections import defaultdict
import numpy as np


In [2]:
def readGz(f):
    for l in gzip.open(f):
        yield eval(l)

In [3]:
#Create training and validation set
user_to_item={}
item_to_user={}
item_to_title={}
split=0
#create training and validation set
for l in readGz("reviews_Musical_Instruments_5.json.gz"):
    user,item,rating,summary = l['reviewerID'],l['asin'],l['overall'],l['summary']
  
    
    if user in user_to_item.keys():
        user_to_item[user].append(item)
    else:
        user_to_item[user]=[item]
        
    if item in item_to_user.keys():
        item_to_user[item].append(user)
    else:
        item_to_user[item] = [user]
for m in readGz("meta_Musical_Instruments.json.gz"):
    item = m['asin']
    if 'title' in m:
        title = m['title']
        if item not in user_to_item.keys():
            item_to_title[item]=[title]
        
    

In [4]:
#a and b are items
def jaccard_similarity(a,b,filtering_type):
    if filtering_type == 'item_based':
        union_count = 0
        total_count = 0
        a_users = item_to_user[a] #list of users who bought a
        #print(a_users)
        b_users = item_to_user[b] #list of users who bought b
       # print(b_users)
        i = len(intersection(a_users, b_users))
        j = len(a_users) + len(b_users) - i
        return i/j
    if filtering_type == 'user_based':
        #a and b will both be users instead of items
        union_count = 0
        total_count = 0
        a_items = user_to_item[a] #list of users who bought a
        #print(a_users)
        b_items = user_to_item[b] #list of users who bought b
       # print(b_users)
        i = len(intersection(a_items, b_items))
        j = len(a_items) + len(b_items) - i
        return i/j
    
                
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3

In [32]:
def recommendation(user, filtering_type):
    if filtering_type == "item_based":
        max_similarity = 0.0

        name = "Name not in metadata"
        other_items = set(item_to_user.keys()) #ALL ITEMS
        print("This users purchased items are: ")
        for item in user_to_item[user]:
            if item in item_to_title.keys():
                print(item_to_title[item])
                other_items.remove(item) #REMOVE ITEMS USER ALREADY BOUGHT TO GIVE NEW RECOMENDATION
        for rec_item in other_items:
            #print("Non-purchased item")
            #print(rec_item)
            for bought_item in user_to_item[user]:
             #   print("Purchased item")
             #   print(bought_item)
                if jaccard_similarity(rec_item, bought_item, 'item_based') > max_similarity:
                    reccomendation = rec_item
                    max_similarity = jaccard_similarity(rec_item, bought_item, 'item_based')

        if reccomendation in item_to_title.keys(): 
            name = item_to_title[reccomendation]
        return name, reccomendation, max_similarity
    
    if filtering_type == "user_based":
        print("This users purchased items are: ")
        for item in user_to_item[user]:
            if item in item_to_title.keys():
                print(item_to_title[item])
        
        #Find most similar users with jaccard similarity
        all_users = set(user_to_item.keys())
        all_users.remove(user)
        max_s = 0.0
        most_similar_users = set()
        for other_user in all_users:
            s = jaccard_similarity(user, other_user, 'user_based' ) 
            if s> max_s:
                most_similar_users = set()
                most_similar_users.add(other_user)
                max_s = s
            if s == max_s:
                most_similar_users.add(other_user) #account for many users having equal similarity
                
        #find items similar users buy that given user did not buy
        other_user_items = [user_to_item[u] for u in most_similar_users]
        other_user_items = set([item for sublist in other_user_items for item in sublist])
        print(other_user_items)
        max_similarity = 0.0
        for i in  user_to_item[user]:
            if i in other_user_items:
                other_user_items.remove(i)
        print(other_user_items)
        for rec_item in other_user_items:
            #print(other_user_items)
            for bought_item in user_to_item[user]:
             #   print(user_to_item[user])
                if jaccard_similarity(rec_item, bought_item, 'item_based') > max_similarity:
                    #print("Recomendation assigned")
                    reccomendation = rec_item
                    max_similarity = jaccard_similarity(rec_item, bought_item, 'item_based')
        if reccomendation in item_to_title.keys(): 
            name = item_to_title[reccomendation]
        return name, reccomendation, max_similarity
    
    
    else:
        return "Filtering type must be 'user_based' or 'item_based.' "
        
        # do item-item similarity on these items
        

In [30]:
recommendation('A2IBPI20UZIR0U', 'item_based')

This users purchased items are: 
['niceEshop Studio Microphone Mic Wind Screen Pop Filter Mask Shied for Speaking Recording']
["D'Addario EXL110 Nickel Wound Electric Guitar Strings, Regular Light, 10-46"]
['Visual Sound One Spot Multi-Plug 5 Cable']
['Hosa IRG6005 6-Inch Guitar Patch Cable']
['Ernie Ball 2222 Hybrid Slinky Nickel Wound Set (9 - 46)']
['Danelectro D-1 Fab Distortion Effects Pedal']
['Protec Guitar Strap with Leather Ends and Pick Pocket, Black']
["D'Addario EJ45 Pro-Arte Nylon Classical Guitar Strings, Normal Tension"]
['Dunlop Formula 65 Guitar Polish &amp; Cleaner, 4 Fluid Ounces']
['Danelectro D-2 Fab Overdrive Effects Pedal']
['On Stage GS7355 Triple Hang It Guitar Stand']
['Planet Waves 9V Power Adapter']
['Fender Treated Polish Cloth']
["BIYANG'S &quot;BABY BOOM&quot; RV-10 TRI-REVERB"]


(['Danelectro D-6 Fab Flange Effects Pedal'], 'B000CBE3GE', 0.25)

In [33]:
recommendation('A3M1PLEYNDEYO8', 'user_based')

This users purchased items are: 
["Monster Standard 100 - 21' Instrument Cable -  Straight to Straight 1/4&quot; plugs"]
['Planet Waves Pro Winder String Winder and Cutter']
["Monster Standard 100 - 21' Instrument Cable - Angled to Straight 1/4&quot; plugs"]
["D'Addario EXP15 Coated Phosphor Bronze Acoustic Guitar Strings, Extra Light, 10-47"]
['On-Stage SM7211 Professional Folding Orchestral Music Stand, Black']
["D'Addario EXL110 Nickel Wound Electric Guitar Strings, Regular Light, 10-46"]
['Hosa CFS606 6 Inch Right Angle FX Pedal Cable - 6 Pack']
['GraphTech PP-1122-00 TUSQ Acoustic Guitar Bridge Pins']
['Hercules GS414B A/G Guitar Stand']
['DUNLOP 220 Jim Chromed Steel Slide']
['Clayton Hula Hottie Guitar Picks 1 Dozen Medium']
['Planet Waves Humidipak Automatic Humidity Control System (For Guitar)']
['Visual Sound One Spot Combo Pack']
['Mighty Bright 51810 Duet2 Music Light']
['Mighty Bright Orchestra Light, with Adapter and Bag']
['TASCAM DR-07MKII Portable Digital Recorder']
["

UnboundLocalError: local variable 'reccomendation' referenced before assignment

In [27]:
recommendation('A1S0HTDO0P4N5V', 'item_based')

This users purchased items are: 
['Hosa Cable MCL105 Series Microphone Cables - 5 Foot']
['Raxxess PTSW25 Rack Screws with Washers (Pack of 25)']
['On Stage MSA9508 Posi-Lok Sidemount Boom']
['Behringer MICROMIX MX400 4-Channel Mixer']
['Behringer ULTRAVOICE XM1800S Dynamic Cardioid Vocal Microphones, 3-Pack']
['American DJ Micro Galaxian Laser Special Effects Lighting']
["Musician's Gear Tripod Mic Stand w/Fixed Boom Black 2-Pack"]


(['On Stage MY550 Microphone Extension Attachment Bar'],
 'B0002ZO3LK',
 0.3333333333333333)

potential issue: there may be many users that are all equally similar to the user in question, do we base reccomendations on this set of similar users or choose one ? (occurs with user 'A1S0HTDO0P4N5V' below)

In [25]:
recommendation('A1S0HTDO0P4N5V', 'user_based')

This users purchased items are: 
['Hosa Cable MCL105 Series Microphone Cables - 5 Foot']
['Raxxess PTSW25 Rack Screws with Washers (Pack of 25)']
['On Stage MSA9508 Posi-Lok Sidemount Boom']
['Behringer MICROMIX MX400 4-Channel Mixer']
['Behringer ULTRAVOICE XM1800S Dynamic Cardioid Vocal Microphones, 3-Pack']
['American DJ Micro Galaxian Laser Special Effects Lighting']
["Musician's Gear Tripod Mic Stand w/Fixed Boom Black 2-Pack"]
{'B0002M3OVI', 'B0006M59M6', 'B004Z8VQ2O', 'B0064RIB9A', 'B004FODY6E', 'B000NJ2TIE', 'B001MSS6CS', 'B001JI2KHS', 'B005TO9EPY', 'B000068O1N'}
<class 'set'>


(['Hosa Cable STP201 TRS To Dual 1/4 Inch Insert Cable - 3 Feet 3 Inches'],
 'B000068O1N',
 0.16666666666666666)

In [58]:
# Using a heap
# > h = []
# >>> heappush(h, (5, 'write code'))
# >>> heappush(h, (7, 'release product'))
# >>> heappush(h, (1, 'write spec'))
# >>> heappush(h, (3, 'create tests'))
# >>> heappop(h)
# (1, 'write spec')
import heapq


In [71]:
def n_recommendations(user, n, filtering_type):
    if filtering_type == 'item_based':
        name = "Name not in metadata"
        best_recs = list()
        titles = []
        other_items = set(item_to_user.keys()) #ALL ITEMS
        print("This users purchased items are: ")
        delete = False
        for item in user_to_item[user]:
            if item in item_to_title.keys():
                print(item_to_title[item]) #print titles of items the user bought
            other_items.remove(item) #remove previously purchased items to get a unique recommendation
        heapq.heapify(best_recs)
        for rec_item in other_items:
            for bought_item in user_to_item[user]:

                similarity = jaccard_similarity(rec_item, bought_item)
                if len(best_recs) < n:
                    heapq.heappush(best_recs, (similarity, rec_item))
                else:
                    smallest = heapq.heappop(best_recs)
                    if smallest[0] >= similarity:
                        heapq.heappush(best_recs, smallest)
                    else:
                        heapq.heappush(best_recs, (similarity, rec_item))
        for rec in best_recs:
            itemid = rec[1] 
            if itemid in item_to_title.keys():
                titles.append(item_to_title[itemid])
            else:
                titles.append(itemid)
        return titles, best_recs
    if filtering_type == 'user_based':
    else:
        return "Filtering type must be 'user_based' or 'item_based' "

In [73]:
n_recommendations('A1S0HTDO0P4N5V', 5)

This users purchased items are: 
['Hosa Cable MCL105 Series Microphone Cables - 5 Foot']
['Raxxess PTSW25 Rack Screws with Washers (Pack of 25)']
['On Stage MSA9508 Posi-Lok Sidemount Boom']
['Behringer MICROMIX MX400 4-Channel Mixer']
['Behringer ULTRAVOICE XM1800S Dynamic Cardioid Vocal Microphones, 3-Pack']
['American DJ Micro Galaxian Laser Special Effects Lighting']
["Musician's Gear Tripod Mic Stand w/Fixed Boom Black 2-Pack"]


([['CBI MLC20 Low Z XLR Microphone Cable, 20 Foot'],
  ['Hosa Cable STP201 TRS To Dual 1/4 Inch Insert Cable - 3 Feet 3 Inches'],
  ['On Stage MY550 Microphone Extension Attachment Bar'],
  ['Chauvet DJ Bank'],
  ['On Stage CM01 Video Camera/Digital Recorder Adapter']],
 [(0.1568627450980392, 'B000VJJQUU'),
  (0.16666666666666666, 'B000068O1N'),
  (0.3333333333333333, 'B0002ZO3LK'),
  (0.25, 'B001E43SK0'),
  (0.2, 'B001GWCC4I')])

In [75]:
n_recommendations('A3M1PLEYNDEYO8', 5)

This users purchased items are: 
["Monster Standard 100 - 21' Instrument Cable -  Straight to Straight 1/4&quot; plugs"]
['Planet Waves Pro Winder String Winder and Cutter']
["Monster Standard 100 - 21' Instrument Cable - Angled to Straight 1/4&quot; plugs"]
["D'Addario EXP15 Coated Phosphor Bronze Acoustic Guitar Strings, Extra Light, 10-47"]
['On-Stage SM7211 Professional Folding Orchestral Music Stand, Black']
["D'Addario EXL110 Nickel Wound Electric Guitar Strings, Regular Light, 10-46"]
['Hosa CFS606 6 Inch Right Angle FX Pedal Cable - 6 Pack']
['GraphTech PP-1122-00 TUSQ Acoustic Guitar Bridge Pins']
['Hercules GS414B A/G Guitar Stand']
['DUNLOP 220 Jim Chromed Steel Slide']
['Clayton Hula Hottie Guitar Picks 1 Dozen Medium']
['Planet Waves Humidipak Automatic Humidity Control System (For Guitar)']
['Visual Sound One Spot Combo Pack']
['Mighty Bright 51810 Duet2 Music Light']
['Mighty Bright Orchestra Light, with Adapter and Bag']
['TASCAM DR-07MKII Portable Digital Recorder']
["

([['Apogee JAM 96k Professional Quality Guitar input for iPad, iPhone and Mac'],
  ["D'Addario EXL120BT Nickel Wound Electric Guitar Strings, Balanced Tension Super Light, 9-40"],
  ["D'Addario EFX165 FlexSteels Bass Guitar Strings, Custom Light, 45-105, Long Scale"],
  ["D'Addario EXL120BT Nickel Wound Electric Guitar Strings, Balanced Tension Super Light, 9-40"],
  ["D'Addario EXL120BT Nickel Wound Electric Guitar Strings, Balanced Tension Super Light, 9-40"]],
 [(0.21052631578947367, 'B00H02C9TG'),
  (0.25, 'B00AQBT3EW'),
  (0.34615384615384615, 'B00HFRXACG'),
  (0.2727272727272727, 'B00AQBT3EW'),
  (0.2619047619047619, 'B00AQBT3EW')])

In [74]:
user_to_item.keys()

dict_keys(['A2IBPI20UZIR0U', 'A14VAT5EAX3D9S', 'A195EZSQDW3E21', 'A2C00NNG1ZQQG2', 'A94QU4C90B1AX', 'A2A039TZMZHH9Y', 'A1UPZM995ZAH90', 'AJNFQI3YR6XJ5', 'A3M1PLEYNDEYO8', 'AMNTZU1YQN1TH', 'A2NYK9KWFMJV4Y', 'A35QFQI0M46LWO', 'A2NIT6BKW11XJQ', 'A1C0O09LOLVI39', 'A17SLR18TUMULM', 'A2PD27UKAD3Q00', 'AKSFZ4G1AXYFC', 'A67OJZLHBBUQ9', 'A2EZWZ8MBEDOLN', 'A1CL807EOUPVP1', 'A1GMWTGXW682GB', 'A2G12DY50U700V', 'A3E0CF25A7LD2', 'A2W3CLAYZLDPTV', 'A398X9POBHK69N', 'AXWB93VKVML6K', 'A2FZ4Z0UFA1OR8', 'AXP9CF1UTFRSU', 'A2CCGGDGZ694CT', 'A27DR1VO079F1V', 'A1LQC225SE8UNI', 'AU9BPT3Y3K6J4', 'A14Z9LAETO21KL', 'A2074KEJGRYJV4', 'A7TRK2GG6BHWD', 'A319B090A2POEB', 'A396ELNTQDFYPO', 'A17CU1D2CWXOHB', 'A1S0HTDO0P4N5V', 'A3DWQW2L39RB1S', 'AQQ91U1U4YKDF', 'AA5TINW2RJ195', 'ABC68JUCPTVOE', 'A3W2E6S24BTXXK', 'A3872Y2XH0YDX1', 'A13A81NN0NRD1S', 'A1EUO0BU72JR7T', 'A1H4WSC8JWS59N', 'A3OXHLG6DIBRW8', 'A12ABV9NU02O29', 'A2U1Z3TZ4P76JB', 'A1L7M2JXN4EZCR', 'A37A41GWQFMK3D', 'A2JK2ITE138P7Z', 'A1DVUFG2QSJ6IK', 'A34VZEFXQJJ

# User-User Reccomendation


In [ ]:
def jaccard_similarity(a,b):
    union_count = 0
    total_count = 0
    a_users = item_to_user[a] #list of users who bought a
    #print(a_users)
    b_users = item_to_user[b] #list of users who bought b
   # print(b_users)
    i = len(intersection(a_users, b_users))
    j = len(a_users) + len(b_users) - i
    return i/j
    
                
def intersection(lst1, lst2): 
    lst3 = [value for value in lst1 if value in lst2] 
    return lst3
